# Lets load in some ephys data and create a plot of it.

In [ ]:
# The following import commands are modules that have helpful functions. 
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import os
print('Modules loaded!')

In [ ]:
# Load in the data using the program provided by OpenEphys. 
# This takes a little bit so wait for the star to disappear to the right
import OpenEphys

cwd = os.getcwd() # Gets your current working directory

pathToData = cwd + '\\data\\Sample_Data\\100_CH1.continuous' # You can change this to look at other channels!
 
data = OpenEphys.load(pathToData)
print('data loaded in!')

## Now that we've loaded the data in. Let's look at what we can do with it.

In [ ]:
# Lets check out what we got
print("The Sampling rate is :", data['header']['sampleRate'])

# There is a problem. The sampling rate is of type string(unicode) even though we need it to be a float...
print("Sample rate is type :", type(data['header']['sampleRate']))

In [ ]:
# So lets change it!
sampleRate = float(data['header']['sampleRate'])
print("The Sampling rate is still :", sampleRate)
print("But the type is now :", type(sampleRate))
print('Notice the precision changed. It\'s now 30000.0!')

In [ ]:
# Compare this to an int?
sampleRate = int(sampleRate)
print('Still the same sampling rate : ', sampleRate)
print('But now the type is : ', type(sampleRate))
print('And once again the precision has changed!')

In [ ]:
# The data is a little longer so we won't print it out. We can look at how many values are stored though.
print('Number of data points :', len(data['data']))
# The timestamps have a different number of values because there is only 1 timestamp per 1024 data samples.
# This is taken care of for you in the next step so don't worry about it.
print('Number of timestamps :', len(data['timestamps']))

dataPerTs = len(data['data'])/len(data['timestamps'])
print('Number of data samples per timestamp', dataPerTs)

### Con is a class that takes in the ephys info and puts it into a easy to use object. Classes are very good ways of holding connected datas together.

Don't worry about what it does. Just know what we get from it.

The ephys information can now be found by the following:

#### conData.data holds the voltage at each timestamp

#### conData.ts holds the timestamps

#### conData.fs holds the sampling rate (how many samples per second)

In [ ]:
class Con:
    def __init__(self, data):
        self.data = data['data'] # Volatage data
        self.fs = float(data['header']['sampleRate']) # Sample Rate
        self.interpTs(data['timestamps']) # Interpolate timestamps for all data points (Only has 1 time stamp for each buffer of 1024 data points)

    def interpTs(self, ts):
        tsStart = ts[0]
        tsEnd = ts[len(ts)-1]
        self.ts = np.linspace(tsStart, tsEnd, len(ts) * 1024) # Creates the array of timestamps
        i = 0
        for t in self.ts:
            self.ts[i] = t - tsStart # Zero out the timestamps, just makes it look better when graphing/explaining
            i = i + 1
        self.tsStart = tsStart # Send the ts start to event data
        
con = Con(data)

## Lets make a very simple plot

In [ ]:
plt.plot(con.ts,con.data, 'r') # This also takes a bit as there is a lot of data.
# Now you have a plot of every voltage for every sample in the recording.
plt.show()

In [ ]:
# Now lets get only the first 10 seconds of data, we'll use the sampling rate we saved before
timeElapsed = int(10 * con.fs) # Feel free to change the 1 to as many seconds as you (needs to be an integer for later)
timeStart = int(0 * con.fs) # Where do you want to start (0 is the start, can change to be as many seconds after as you want)
timeEnd = timeElapsed + timeStart # We need what timestamp the end is. It is the start index + how many have elapsed over 10 sec
plt.plot(con.ts[timeStart:timeEnd], con.data[timeStart:timeEnd], 'r') # [startIndex:endIndex]
plt.show()

# Now time for you to try some programming yourself.

## First challenge

Use a for loop to iterate through the data points and plot a vertical line when the voltage goes above 500.

In [ ]:
timeElapsed = int(10 * con.fs) # Feel free to change the 1 to as many seconds as you (needs to be an integer for later)
timeStart = int(0 * con.fs) # Where do you want to start (0 is the start, can change to be as many seconds after as you want)
timeEnd = timeElapsed + timeStart # We need what timestamp the end is. It is the start index + how many have elapsed over 10 sec
# Put code here!
# use matplotlib to create a vertical line (hint: use google!)

plt.plot(con.ts[timeStart:timeEnd], con.data[timeStart:timeEnd], 'r') 
plt.show()

## Second challenge

Plot a horizontal line at the average value for the data.

In [ ]:
# Put code here!

plt.plot(con.ts[timeStart:timeEnd], con.data[timeStart:timeEnd], 'r') 
# use matplotlib to create a horizontal line (hint: use google!)

plt.show()

## Third challenge

Use the butter function from scipys signal library to create a bandpass filter to extract only the theta band (4-8hz)! If you want, writing a few functions may help you and be good practice. 

Use google to understand how to use butter. Examples are very helpful!

In [ ]:
# If you want to use functions write them here.

In [ ]:
from scipy import signal 
# Hints!
# butter function can be accessed by signal.butter
# You will also need to use lfilter function which is included in the signal library. 
# Use a filter order of 2
# You will also need to normalize your band that you want to extract
#   by the nyquist frequency


# Put code here

plt.plot(con.ts[timeStart:timeEnd], filteredData[timeStart:timeEnd], 'r')
plt.show()